In [6]:
import json
import os
from pathlib import Path

def extractData(filepath):
    with open (filepath) as f:
        data = json.loads(f.read())
    required_data = {}
    episodes_data = data["episodes"]
    for episode in episodes_data:
        scenes_data = episode["scenes"]
        for scene in scenes_data:
            dialogue_list = []
            count_dict = {"Total":0,"Ross Geller":0, "Rachel Green":0, "Monica Geller":0, "Chandler Bing":0, "Joey Tribbiani":0,"Phoebe Buffay":0}
            for utterance in scene["utterances"]:
                if len(utterance["speakers"]) > 0:
                    temp_list = [",".join(utterance["speakers"]),utterance["transcript"]]
                    dialogue_list.append(temp_list)
                    if temp_list[0] in list(count_dict.keys()):
                        count_dict[temp_list[0]]+=1
                        count_dict["Total"]+=1
            required_data[scene["scene_id"]] = {"dialogues":dialogue_list,"count":count_dict}
    return required_data

def getSequenceWiseUtterences(filepath):
    with open (filepath) as f:
        data = json.loads(f.read())
    dialogue_list = []
    episodes_data = data["episodes"]
    for episode in episodes_data:
        scenes_data = episode["scenes"]
        for scene in scenes_data:
            for utterance in scene["utterances"]:
                if len(utterance["speakers"]) > 0 and len(utterance["transcript"].split(" ")) > 5:
                    dialogue_list.append(utterance["transcript"])
    
    return dialogue_list
                    
    
        

filepath = "json/Seasons/"
filenames = os.listdir("json/Seasons/")
storePath = "Data/"

for i in filenames:    
    data_dict = extractData(os.path.join(filepath,i))
    json_save_path = Path(os.path.join(storePath,"Dataset",i.split(".")[0][8:]+".json"))
    json_save_path.write_text(json.dumps(data_dict))

    dialogue_list = getSequenceWiseUtterences(os.path.join(filepath,i))
    f = open(os.path.join(storePath,"Sequence_Utterences",i.split(".")[0][8:]+".txt"), "w")
    for dialog in dialogue_list:
        f.write(dialog+"\n")
    f.close()


{'dialogues': [['Monica Geller', "There's nothing to tell! He's just some guy I work with!"], ['Joey Tribbiani', "C'mon, you're going out with the guy! There's gotta be something wrong with him!"], ['Chandler Bing', 'All right Joey, be nice. So does he have a hump? A hump and a hairpiece?'], ['Phoebe Buffay', 'Wait, does he eat chalk?'], ['Phoebe Buffay', "Just, 'cause, I don't want her to go through what I went through with Carl- oh!"], ['Monica Geller', "Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex."], ['Chandler Bing', 'Sounds like a date to me.'], ['Chandler Bing', "Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked."], ['#ALL#', 'Oh, yeah. Had that dream.'], ['Chandler Bing', "Then I look down, and I realize there's a phone... there."], ['Joey Tribbiani', 'Instead of...?'], ['Chandler Bing', "That's right."], ['Joey Tribbiani', 'Never had that dream.'], [